In [1]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

<IPython.core.display.Javascript object>

In [2]:
keyList = ["Title",
                "Author",
                "About_author",
                "Foreword",
                "Description",
                "Image",
                "Genres", 
                "Series",
                "Order",
                "Pages",
                "Format",
                "First_published",
                "Rating",
                "Review_count"]



In [7]:
books = []
reviews_link = []
series_all = []
authors_link = []
genres_link = []

file = open('links1.txt', "r")
lines = file.readlines()

count = 0
for line in lines:
    book = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    book_info = line.strip()
    header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    response = requests.get(book_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('books.html', 'w', encoding="utf-8") as f:
        f.write(page_contents)

    # read it html page to scrape information
    with open('books.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_book = BeautifulSoup(html_source, 'html.parser')

    for br in doc_book.select("br"):
        br.replace_with("\n")
    
    # get all the company name in the page
    book_name = doc_book.find('h1', attrs={'data-testid': 'bookTitle'})
    #print(book_name)
    book['Title'] = book_name.text.strip()
    # for the first company name, get the exact name

    book_author = doc_book.find('span', attrs={'data-testid': 'name'})
    #print(book_author)
    book['Author'] = book_author.text.strip()
    # for the first company name, get the exact name

    book_rating = doc_book.find('div', class_= 'RatingStatistics__rating')
    #print(book_rating)
    book['Rating'] = book_rating.text.strip()

    review_count = doc_book.find('span', attrs={'data-testid': 'reviewsCount'})
    #print(review_count)
    review_count = review_count.text.strip()
    review_count = ''.join(x for x in review_count if x.isdigit())
    book['Review_count'] = review_count
    # for the first company name, get the exact name

    book_description = doc_book.find('div', attrs={'data-testid': 'description'})
    #print(book_description)
    br = book_description.find('b')
    if br != None:
        book['Foreword'] = br.text.strip()
        book_description.find('b').decompose()

    book_description = doc_book.find('div', attrs={'data-testid': 'description'})
    des = book_description.text.strip()
    book['Description'] = des

    book_image = doc_book.find('img', class_='ResponsiveImage')
    book['Image'] = book_image['src']

    genres = []
    for type in doc_book.find_all('span', class_='BookPageMetadataSection__genreButton'):
        genre = type.text.strip()
        a = type.find('a')
        i = a.get('href')
        genres_link.append(i)
        genres.append(genre)

    genre = ", ".join(genres)
    book['Genres'] = genre

    book_series = doc_book.find('h3', class_='Text Text__title3 Text__italic Text__regular Text__subdued')
    if book_series != None:
        a = book_series.find('a')
        series_link = a.get('href')
        series_all.append(series_link)
        series = book_series.text.strip().split('#')
        book['Series'] = series[0]
        if len(series) != 1:
            book['Order'] = series[1]

    book_form = doc_book.find('p', attrs={'data-testid': 'pagesFormat'})
    #print(book_form)
    book_form = book_form.text.strip()
    book_form = book_form.split(' ')
    book['Pages'] = book_form[0]
    book['Format'] = book_form[2]
    # for the first company name, get the exact name

    book_date = doc_book.find('p', attrs={'data-testid': 'publicationInfo'})
    book_date = book_date.text.strip()
    book_date = book_date.split(' ')
    book_date.remove('First')
    book_date.remove('published')
    book['First_published'] = ' '.join(book_date)

    author_info = doc_book.find('div', class_='FeaturedPerson__infoPrimary')
    author_info = author_info.find('a')
    author_info = author_info.get('href')
    authors_link.append(author_info)

    reviews = []
    reviews = doc_book.find_all('span', class_='Text Text__body3')
    for i in reviews:
        review = i.find('a')
        if review is None:
            continue
        else:
            review = str(review.get('href'))
            reviews_link.append(review)

    author_des = doc_book.find_all('div', class_='DetailsLayoutRightParagraph__widthConstrained')
    author_des = author_des[1]
    author_des = author_des.text.strip()
    book['About_author'] = author_des
    print(book)
    books.append(book)

Line1: https://www.goodreads.com/book/show/2767052-the-hunger-games
{'Title': 'The Hunger Games', 'Author': 'Suzanne Collins', 'About_author': 'Librarian Note: There is more than one author in the Goodreads database with this name.\n\nSince 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\n\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\n\nThinking one day about Alice in Wonderland, she was struck by how pastoral the setting must seem to kids w

In [8]:
reviewFile = open("reviews.txt", "a") 
for i in reviews_link:
    reviewFile.write(i+"\n")        
reviewFile.close()

seriesFile = open('series.txt', "a")
for i in series_all:
    seriesFile.write(i+"\n")        
seriesFile.close()

authorFile = open("authors.txt", "a") 
for i in authors_link:
    authorFile.write(i+"\n")        
authorFile.close()

genreFile = open('genres.txt', "a")
for i in genres_link:
    genreFile.write(i+"\n")        
genreFile.close()

In [5]:
books_df = pd.DataFrame(data=books)
books_df

,Title,Author,About_author,Foreword,Description,Image,Genres,Series,Order,Pages,Format,First_published,Rating,Review_count
0,The Hunger Games,Suzanne Collins,Librarian Note: There is more than one author ...,"Could you survive on your own in the wild, wit...",In the ruins of a place once known as North Am...,https://images-na.ssl-images-amazon.com/images...,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",The Hunger Games,1,374,Hardcover,"September 14, 2008",4.33,205790
1,Harry Potter and the Order of the Phoenix,J.K. Rowling,See also: Robert Galbraith\nAlthough she write...,None,Harry Potter is about to start his fifth year ...,https://images-na.ssl-images-amazon.com/images...,"Young Adult, Fiction, Magic, Childrens, Audiob...",Harry Potter,5,912,Paperback,"June 21, 2003",4.50,60268
2,Pride and Prejudice,Jane Austen,Jane Austen was an English novelist whose work...,None,"Since its immediate success in 1813, Pride and...",https://images-na.ssl-images-amazon.com/images...,"Classics, Fiction, Romance, Historical Fiction...",None,None,279,Paperback,"January 28, 1813",4.28,107767
3,To Kill a Mockingbird,Harper Lee,"Harper Lee, known as Nelle, was born in the Al...",None,The unforgettable novel of a childhood in a sl...,https://images-na.ssl-images-amazon.com/images...,"Classics, Fiction, Historical Fiction, School,...",To Kill a Mockingbird,None,323,Paperback,"July 11, 1960",4.26,113870
4,The Book Thief,Markus Zusak,"Markus Zusak is the author of five books, incl...",None,Librarian's note: An alternate cover edition c...,https://images-na.ssl-images-amazon.com/images...,"Historical Fiction, Fiction, Young Adult, Hist...",None,None,592,Hardcover,"March 1, 2005",4.39,140648
5,Twilight,Stephenie Meyer,Stephenie Meyer is the author of the bestselli...,Deeply seductive and extraordinarily suspensef...,About three things I was absolutely positive.\...,https://images-na.ssl-images-amazon.com/images...,"Fantasy, Young Adult, Romance, Fiction, Vampir...",The Twilight Saga,1,498,Paperback,"October 5, 2005",3.64,126469
6,Animal Farm,George Orwell,"Eric Arthur Blair, better known by his pen nam...",None,Librarian's note: There is an Alternate Cover ...,https://images-na.ssl-images-amazon.com/images...,"Classics, Fiction, Dystopia, Fantasy, School, ...",None,None,128,Mass,"August 17, 1945",3.98,90229
7,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,"John Ronald Reuel Tolkien: writer, artist, sch...",None,"This four-volume, boxed set contains J.R.R. To...",https://images-na.ssl-images-amazon.com/images...,"Fantasy, Fiction, Classics, Adventure, Science...",The Lord of the Rings,0-3,1728,Mass,"January 1, 1954",4.61,2202
8,The Chronicles of Narnia,C.S. Lewis,Librarian Note: There is more than one author ...,None,Librarian note: An alternate cover for this ed...,https://images-na.ssl-images-amazon.com/images...,"Fantasy, Classics, Fiction, Young Adult, Child...",The Chronicles of Narnia (Publication Order),1–7,767,Paperback,"January 1, 1956",4.27,12251
9,The Fault in Our Stars,John Green,Librarian Note: There is more than one author ...,None,Despite the tumor-shrinking medical miracle th...,https://images-na.ssl-images-amazon.com/images...,"Young Adult, Fiction, Contemporary, Realistic ...",None,None,313,Hardcover,"January 10, 2012",4.14,176178


In [6]:
books_df.to_csv('books.csv', index = True)